<a href="https://colab.research.google.com/github/nmansour67/skills-introduction-to-github/blob/main/In_Silo_Trial_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ========================================================
# SECTION 4: THE IN-SILICO TRIAL ANALYZER
# ========================================================

import pandas as pd
import numpy as np
from google.colab import files
import io

# --- STEP 1: UPLOAD YOUR SYNTHETIC DATA ---
print("📂 STEP 1: Upload the 'mobility_data.csv' file:")
uploaded_mob = files.upload()
filename_mob = list(uploaded_mob.keys())[0]
df_mobility = pd.read_csv(io.BytesIO(uploaded_mob[filename_mob]))

print("\n📂 STEP 2: Upload the 'bedrest_data.csv' file:")
uploaded_bed = files.upload()
filename_bed = list(uploaded_bed.keys())[0]
df_bedrest = pd.read_csv(io.BytesIO(uploaded_bed[filename_bed]))

# Add a label column
df_mobility['Protocol'] = 'Early Mobility'
df_bedrest['Protocol'] = 'Bedrest'

# Combine them for analysis
df_all = pd.concat([df_mobility, df_bedrest])

# Convert 'Outcome_Delirium' to numerical (1 for 'Yes', 0 for 'No')
df_mobility['Outcome_Delirium'] = df_mobility['Outcome_Delirium'].map({'Yes': 1, 'No': 0})
df_bedrest['Outcome_Delirium'] = df_bedrest['Outcome_Delirium'].map({'Yes': 1, 'No': 0})

# --- STEP 2: THE "NAIVE" COMPARISON (The Trap) ---
print("\n" + "="*50)
print("🧐 THE NAIVE ANALYSIS (What the chart says)")
print("="*50)

delirium_mob = df_mobility['Outcome_Delirium'].mean()
delirium_bed = df_bedrest['Outcome_Delirium'].mean()

print(f"Delirium Rate (Mobility): {delirium_mob*100:.1f}%")
print(f"Delirium Rate (Bedrest):  {delirium_bed*100:.1f}%")

if delirium_bed > delirium_mob:
    print("👉 CONCLUSION: Bedrest looks terrible! It causes delirium!")
else:
    print("👉 CONCLUSION: No difference found.")

# --- STEP 3: THE "DETECTIVE" WORK (Finding the Bias) ---
print("\n" + "="*50)
print("🕵️ THE DETECTIVE WORK (Why the chart is lying)")
print("="*50)

avg_sofa_mob = df_mobility['SOFA_Score'].mean()
avg_sofa_bed = df_bedrest['SOFA_Score'].mean()

print(f"Average Sickness (SOFA) - Mobility Group: {avg_sofa_mob:.1f}")
print(f"Average Sickness (SOFA) - Bedrest Group:  {avg_sofa_bed:.1f}")

diff = avg_sofa_bed - avg_sofa_mob
print(f"\n🚨 THE REVEAL: The Bedrest patients were {diff:.1f} points SICKER on average.")
print("You cannot compare these groups directly. You just found the Selection Bias.")

# --- STEP 4: THE "MANUAL TWIN MATCH" ---
# Let's find one pair to compare fairly
print("\n" + "="*50)
print("👯 THE MANUAL TWIN MATCH")
print("="*50)

# Try to find a match based on SOFA score
target_sofa = int(df_mobility['SOFA_Score'].median())
print(f"Looking for Twins with SOFA Score: {target_sofa}...")

twin_mob = df_mobility[df_mobility['SOFA_Score'] == target_sofa]
twin_bed = df_bedrest[df_bedrest['SOFA_Score'] == target_sofa]

if not twin_mob.empty and not twin_bed.empty:
    print(f"\nFound Twins!")
    print(f"Mobility Patient (SOFA {target_sofa}): Delirium = {twin_mob.iloc[0]['Outcome_Delirium']}")
    print(f"Bedrest Patient  (SOFA {target_sofa}): Delirium = {twin_bed.iloc[0]['Outcome_Delirium']}")
    print("\nCompare THESE two outcomes for the real truth.")
else:
    print("\nNo exact twins found (Bias is too strong!). This proves why we need advanced math!")

📂 STEP 1: Upload the 'mobility_data.csv' file:


Saving bedrest_group.csv to bedrest_group (1).csv

📂 STEP 2: Upload the 'bedrest_data.csv' file:


Saving mobility_group.csv to mobility_group (1).csv

🧐 THE NAIVE ANALYSIS (What the chart says)
Delirium Rate (Mobility): 70.0%
Delirium Rate (Bedrest):  50.0%
👉 CONCLUSION: No difference found.

🕵️ THE DETECTIVE WORK (Why the chart is lying)
Average Sickness (SOFA) - Mobility Group: 10.6
Average Sickness (SOFA) - Bedrest Group:  6.1

🚨 THE REVEAL: The Bedrest patients were -4.5 points SICKER on average.
You cannot compare these groups directly. You just found the Selection Bias.

👯 THE MANUAL TWIN MATCH
Looking for Twins with SOFA Score: 10...

Found Twins!
Mobility Patient (SOFA 10): Delirium = 0
Bedrest Patient  (SOFA 10): Delirium = 1

Compare THESE two outcomes for the real truth.
